In [77]:
!pip install numpy 
!pip install pandas
import re
import numpy as np
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [78]:
df_trn=pd.read_csv("spatiotemporal_tst_data.csv")
df_trn.head

/tmp/ipykernel_147691/2119717897.py:1: DtypeWarning: Columns (8,11,13,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_trn=pd.read_csv("spatiotemporal_tst_data.csv")


<bound method NDFrame.head of             STATION                 DATE   LATITUDE  LONGITUDE  ELEVATION  \
0       42369099999  1999-03-12T22:00:00  26.760594  80.889339     124.96   
1       42369099999  2007-03-31T05:30:00  26.760594  80.889339     124.96   
2       42260099999  1977-02-19T20:30:00  27.155831  77.960892     167.94   
3       42634099999  1975-01-22T23:30:00  23.287828  69.670147      81.68   
4       42170099999  2019-06-25T20:30:00  28.250000  74.916667     291.00   
...             ...                  ...        ...        ...        ...   
103863  42379099999  2012-10-15T17:30:00  26.739708  83.449708      78.94   
103864  42273099999  1994-06-01T17:30:00  27.566667  81.600000     124.00   
103865  42452099999  1990-12-29T08:30:00  25.160219  75.845631     273.10   
103866  42503099999  2020-03-13T14:30:00  25.033333  88.133333      31.00   
103867  42165099999  1980-07-20T17:30:00  28.000000  73.300000     224.00   

                 NAME REPORT_TYPE  SOURCE Hou

In [79]:
##remove the nan value
# Add targets to same df
# df_trn['TARGETS'] = pd.read_csv('spatiotemporal_trn_targets.csv', names=['index', 'TARGETS'])['TARGETS']
df_clean = df_trn.dropna(axis=1, how='all')
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103868 entries, 0 to 103867
Data columns (total 25 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   STATION                             103868 non-null  int64  
 1   DATE                                103868 non-null  object 
 2   LATITUDE                            103868 non-null  float64
 3   LONGITUDE                           103868 non-null  float64
 4   ELEVATION                           103868 non-null  float64
 5   NAME                                103868 non-null  object 
 6   REPORT_TYPE                         103868 non-null  object 
 7   SOURCE                              103868 non-null  int64  
 8   HourlyAltimeterSetting              24086 non-null   object 
 9   HourlyDewPointTemperature           101938 non-null  object 
 10  HourlyDryBulbTemperature            102611 non-null  object 
 11  HourlyPrecipitation       

In [80]:
# df_clean.to_csv('clean.csv', index=False)

In [81]:
def get_date_dict(date_str):
    reg = re.compile(r"^(?P<yr>\d{4})-(?P<mon>\d{2})-(?P<day>\d{2})T(?P<hr>\d{2}):(?P<min>\d{2}):(?P<sec>\d{2})$")
    m = reg.match(date_str)
    if m:
        return m.groupdict()
    else:
        raise ValueError('The date string does not match the expected format.')

In [82]:
# Convert the 'DATE' column to datetime objects
df_clean['DATE'] = pd.to_datetime(df_clean['DATE'])

# Extract the date components directly without a loop
df_clean['YEAR'] = df_clean['DATE'].dt.year
df_clean['MONTH'] = df_clean['DATE'].dt.month
df_clean['DAY'] = df_clean['DATE'].dt.day
df_clean['HOUR'] = df_clean['DATE'].dt.hour
df_clean['MINUTES'] = df_clean['DATE'].dt.minute
df_clean['SECONDS'] = df_clean['DATE'].dt.second


/tmp/ipykernel_147691/1878784476.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['DATE'] = pd.to_datetime(df_clean['DATE'])
/tmp/ipykernel_147691/1878784476.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['YEAR'] = df_clean['DATE'].dt.year
/tmp/ipykernel_147691/1878784476.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [83]:
df_clean.drop(["HourlySkyConditions","SOURCE","LATITUDE","LONGITUDE","REPORT_TYPE","ShortDurationPrecipitationValue150","REM", "DATE","NAME", "HourlyPrecipitation", "HourlyPressureChange", "HourlyWindGustSpeed"], axis=1,inplace=True)



/tmp/ipykernel_147691/4203075989.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop(["HourlySkyConditions","SOURCE","LATITUDE","LONGITUDE","REPORT_TYPE","ShortDurationPrecipitationValue150","REM", "DATE","NAME", "HourlyPrecipitation", "HourlyPressureChange", "HourlyWindGustSpeed"], axis=1,inplace=True)


In [84]:
encode_cols = ["HourlyPresentWeatherType"]
cat_vector_1 = df_clean[encode_cols[0]].value_counts().index
cat_vector_1

Index(['||HZ', '||DU', '||FG', '||FU', '||RA', '||TS', '||DZ', '||SHRA TS',
       '||SHRA', '||HZ DU', '||SH', '||RA TS', '||HZ RA', '||TS RA', '||FG RA',
       '||HZ DZ', '||FU HZ', '||FG FG', '||DZ DZ', '||RA RA', '||s',
       '||TS SHRA TS', '||SHSN', '||TS TS', '||FG DZ', '||HZ SHRA', '||FU FG',
       '||DU DZ', '||FG SHRA', '||HZ TS', '||DU DU', '||FZDZ', '|FU |HZ',
       '||DZ RA', '||DZ TS', '||DU RA', '|FU |', '|TS+HAIL |', '||FG DU',
       '||DU SHRA TS', '||FC', '||DU TS'],
      dtype='object')

In [85]:
cat_vector_1 = list(cat_vector_1)
enc_vector_1 = [cat_vector_1.index(cat) for cat in cat_vector_1]
enc_vector_1 = [i if i<7 else 7 for i in enc_vector_1]
encode_dict_1 = dict(zip(cat_vector_1, enc_vector_1))
encode_dict_1



# encoded from 0 to 8, 0 to 6 for major categories, 7 for minor, 8 for invalid or missing
encode_dict_1['|HZ |'] = 0
encode_dict_1['|DZ |'] = 6
encode_dict_1['|FU |'] = 3
encode_dict_1['||s'] = 8
encode_dict_1[np.nan] = 8




colm = df_clean[encode_cols[0]].copy()
colm = pd.Series(map(lambda i: encode_dict_1.get(i, 8), colm))
colm.value_counts()

df_clean[encode_cols[0]] = colm



/tmp/ipykernel_147691/347485450.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[encode_cols[0]] = colm


In [86]:
df_clean.columns

Index(['STATION', 'ELEVATION', 'HourlyAltimeterSetting',
       'HourlyDewPointTemperature', 'HourlyDryBulbTemperature',
       'HourlyPresentWeatherType', 'HourlyPressureTendency',
       'HourlyRelativeHumidity', 'HourlySeaLevelPressure',
       'HourlyStationPressure', 'HourlyWetBulbTemperature',
       'HourlyWindDirection', 'HourlyWindSpeed', 'YEAR', 'MONTH', 'DAY',
       'HOUR', 'MINUTES', 'SECONDS'],
      dtype='object')

In [87]:

def get_numeric(string):
    try:
        return float(string)
    except:
        if type(string) is str:
            reg = re.compile(r"(?P<num>-?\d+)(?P<dec>\.\d+)?")
            m = reg.match(string)
            if m:
                num = m.groupdict().get('num')
                dec = m.groupdict().get('dec')
                if dec:
                    return float(num+dec)
                else:
                    return float(num)
        else:
            return np.nan



In [88]:
df_clean.HourlyAltimeterSetting.value_counts()

29.91     1015
29.94      984
29.97      946
30         913
29.62      901
29.59      893
29.53      879
29.56      834
29.88      796
29.85      793
29.5       783
29.65      745
29.68      720
29.47      710
29.71      686
29.74      682
29.83      674
29.8       673
29.77      662
30.03      622
29.44      530
30.06      469
29.41      409
30.09      326
29.97      310
29.94      309
29.91      289
29.59      285
30.0       269
29.62      263
29.53      258
29.88      254
29.65      248
29.83      243
29.5       242
29.56      234
29.68      233
29.47      230
29.38      229
29.71      226
29.85      218
30.03      207
29.8       194
30.12      192
29.74      192
29.77      181
29.44      178
30.06      152
29.41      107
30.09      106
30.15      101
29.35       96
29.38       70
30.12       55
29.32       50
30.18       47
30.15       35
29.35       28
30.21       20
30.18       13
29.32       12
29.26       10
29.29        9
30.24        8
30.21        8
29.21        4
29.29     

In [89]:
df_clean['HourlyAltimeterSetting'] = df_clean['HourlyAltimeterSetting'].apply(get_numeric)
df_clean['HourlyDewPointTemperature'] = df_clean['HourlyDewPointTemperature'].apply(get_numeric)
df_clean['HourlyDryBulbTemperature'] = df_clean['HourlyDryBulbTemperature'].apply(get_numeric)
df_clean['HourlySeaLevelPressure'] = df_clean['HourlySeaLevelPressure'].apply(get_numeric)
df_clean['HourlyStationPressure'] = df_clean['HourlyStationPressure'].apply(get_numeric)
df_clean['HourlyWindSpeed'] = df_clean['HourlyWindSpeed'].apply(get_numeric)
df_clean['HourlyWindDirection'] = df_clean['HourlyWindDirection'].apply(get_numeric)
# df_clean['TARGETS'] = df_clean['TARGETS'].apply(get_numeric)


/tmp/ipykernel_147691/3218609761.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['HourlyAltimeterSetting'] = df_clean['HourlyAltimeterSetting'].apply(get_numeric)
/tmp/ipykernel_147691/3218609761.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['HourlyDewPointTemperature'] = df_clean['HourlyDewPointTemperature'].apply(get_numeric)
/tmp/ipykernel_147691/3218609761.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [90]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103868 entries, 0 to 103867
Data columns (total 19 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   STATION                    103868 non-null  int64  
 1   ELEVATION                  103868 non-null  float64
 2   HourlyAltimeterSetting     24086 non-null   float64
 3   HourlyDewPointTemperature  101938 non-null  float64
 4   HourlyDryBulbTemperature   102611 non-null  float64
 5   HourlyPresentWeatherType   103868 non-null  int64  
 6   HourlyPressureTendency     65284 non-null   float64
 7   HourlyRelativeHumidity     101881 non-null  float64
 8   HourlySeaLevelPressure     74613 non-null   float64
 9   HourlyStationPressure      2732 non-null    float64
 10  HourlyWetBulbTemperature   2715 non-null    float64
 11  HourlyWindDirection        96079 non-null   float64
 12  HourlyWindSpeed            102673 non-null  float64
 13  YEAR                       10

In [91]:
# df_clean.HourlyPresentWeatherType.value_counts()

In [92]:
pd.set_option('display.max_rows', 100)

In [93]:
# df = df_clean[df_clean['TARGETS'] < 40]
# print(df.head())

In [94]:
df=df_clean
df.drop("SECONDS",axis=1,inplace=True)

/tmp/ipykernel_147691/4229671364.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop("SECONDS",axis=1,inplace=True)


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103868 entries, 0 to 103867
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   STATION                    103868 non-null  int64  
 1   ELEVATION                  103868 non-null  float64
 2   HourlyAltimeterSetting     24086 non-null   float64
 3   HourlyDewPointTemperature  101938 non-null  float64
 4   HourlyDryBulbTemperature   102611 non-null  float64
 5   HourlyPresentWeatherType   103868 non-null  int64  
 6   HourlyPressureTendency     65284 non-null   float64
 7   HourlyRelativeHumidity     101881 non-null  float64
 8   HourlySeaLevelPressure     74613 non-null   float64
 9   HourlyStationPressure      2732 non-null    float64
 10  HourlyWetBulbTemperature   2715 non-null    float64
 11  HourlyWindDirection        96079 non-null   float64
 12  HourlyWindSpeed            102673 non-null  float64
 13  YEAR                       10

In [96]:
df.sort_values(['STATION', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTES'], inplace = True)

/tmp/ipykernel_147691/1408955147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(['STATION', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTES'], inplace = True)


In [97]:
df.head()

,STATION,ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPresentWeatherType,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindSpeed,YEAR,MONTH,DAY,HOUR,MINUTES
71832,42030099999,3066.0,NaN,NaN,-25.0,8,NaN,NaN,29.80,NaN,NaN,0.0,0.0,1944,1,2,8,30
73520,42030099999,3066.0,NaN,-17.0,-14.0,8,NaN,86.0,29.80,NaN,NaN,0.0,0.0,1944,1,4,8,30
66567,42030099999,3066.0,NaN,NaN,6.0,8,NaN,NaN,29.76,NaN,NaN,0.0,0.0,1944,1,5,8,30
56160,42030099999,3066.0,NaN,14.0,15.0,8,NaN,95.0,29.73,NaN,NaN,0.0,0.0,1944,1,9,8,30
85363,42030099999,3066.0,NaN,-22.0,-20.0,8,NaN,90.0,29.80,NaN,NaN,0.0,0.0,1944,1,10,8,30


In [98]:


df.reset_index(inplace = True, drop=True)



In [99]:
df.head()

,STATION,ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPresentWeatherType,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindSpeed,YEAR,MONTH,DAY,HOUR,MINUTES
0,42030099999,3066.0,NaN,NaN,-25.0,8,NaN,NaN,29.80,NaN,NaN,0.0,0.0,1944,1,2,8,30
1,42030099999,3066.0,NaN,-17.0,-14.0,8,NaN,86.0,29.80,NaN,NaN,0.0,0.0,1944,1,4,8,30
2,42030099999,3066.0,NaN,NaN,6.0,8,NaN,NaN,29.76,NaN,NaN,0.0,0.0,1944,1,5,8,30
3,42030099999,3066.0,NaN,14.0,15.0,8,NaN,95.0,29.73,NaN,NaN,0.0,0.0,1944,1,9,8,30
4,42030099999,3066.0,NaN,-22.0,-20.0,8,NaN,90.0,29.80,NaN,NaN,0.0,0.0,1944,1,10,8,30


In [100]:
# df.to_csv('clean_data.csv', index=False)

In [101]:
!pip install tqdm
from tqdm import tqdm
groups = []
start = 0
for idx in tqdm(range(1, len(df))):
    if df.STATION[idx-1] == df.STATION[idx]:
        continue
    else:
        end = idx-1
        groups.append((start, end))
        start = idx

Defaulting to user installation because normal site-packages is not writeable


100%|██████████| 103867/103867 [00:00<00:00, 151004.28it/s]


In [102]:
groups.append((start, len(df)-1))
len(groups) == df.STATION.nunique()
groups


[(0, 219),
 (220, 1761),
 (1762, 1762),
 (1763, 7434),
 (7435, 9304),
 (9305, 9977),
 (9978, 19572),
 (19573, 20049),
 (20050, 20211),
 (20212, 22633),
 (22634, 24217),
 (24218, 24565),
 (24566, 25312),
 (25313, 60300),
 (60301, 71413),
 (71414, 71725),
 (71726, 82794),
 (82795, 82986),
 (82987, 83304),
 (83305, 83305),
 (83306, 87401),
 (87402, 88896),
 (88897, 89220),
 (89221, 89807),
 (89808, 90122),
 (90123, 103522),
 (103523, 103867)]

In [103]:
df.shape

(103868, 18)

In [104]:
df.head

<bound method NDFrame.head of             STATION  ELEVATION  HourlyAltimeterSetting  \
0       42030099999     3066.0                     NaN   
1       42030099999     3066.0                     NaN   
2       42030099999     3066.0                     NaN   
3       42030099999     3066.0                     NaN   
4       42030099999     3066.0                     NaN   
...             ...        ...                     ...   
103863  42672099999      440.0                     NaN   
103864  42672099999      440.0                     NaN   
103865  42672099999      440.0                     NaN   
103866  42672099999      440.0                     NaN   
103867  42672099999      440.0                     NaN   

        HourlyDewPointTemperature  HourlyDryBulbTemperature  \
0                             NaN                     -25.0   
1                           -17.0                     -14.0   
2                             NaN                       6.0   
3                    

In [105]:
df.isna().sum()

STATION                           0
ELEVATION                         0
HourlyAltimeterSetting        79782
HourlyDewPointTemperature      1930
HourlyDryBulbTemperature       1257
HourlyPresentWeatherType          0
HourlyPressureTendency        38584
HourlyRelativeHumidity         1987
HourlySeaLevelPressure        29255
HourlyStationPressure        101136
HourlyWetBulbTemperature     101153
HourlyWindDirection            7789
HourlyWindSpeed                1195
YEAR                              0
MONTH                             0
DAY                               0
HOUR                              0
MINUTES                           0
dtype: int64

In [106]:
cols = ['HourlyAltimeterSetting', 'HourlyDewPointTemperature', 'HourlyDryBulbTemperature','HourlyPresentWeatherType ', 'HourlyPressureTendency', 
       'HourlyRelativeHumidity', 'HourlySeaLevelPressure', 'HourlyStationPressure', 'HourlyWetBulbTemperature', 
       'HourlyWindDirection', 'HourlyWindSpeed']
imp_df = pd.DataFrame(columns=cols)
imp_df.head()

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPresentWeatherType,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindSpeed


In [107]:
import pandas as pd
from sklearn.impute import KNNImputer
from tqdm import tqdm

# Create an instance of KNNImputer
imputer = KNNImputer(n_neighbors=3,weights='distance',metric='nan_euclidean')  

# Iterate over each group and apply imputation
for start, end in tqdm(groups):
    temp = df.loc[start:end, 'HourlyAltimeterSetting':'HourlyWindSpeed']

    print(temp.shape) 
    # print(temp)
    temp2 = imputer.fit_transform(temp)
    temp2= pd.DataFrame(temp2)

    column_mapping = {
    0: 'HourlyAltimeterSetting',
    1: 'HourlyDewPointTemperature',
    2: 'HourlyDryBulbTemperature',
    3: 'HourlyPresentWeatherType',
    4: 'HourlyPressureTendency',
    5: 'HourlyRelativeHumidity',
    6: 'HourlySeaLevelPressure',
    7: 'HourlyStationPressure',
    8: 'HourlyWetBulbTemperature',
    9: 'HourlyWindDirection',
    10: 'HourlyWindSpeed'
    }

    temp2.rename(columns=column_mapping, inplace=True)

    imp_df = pd.concat([imp_df, temp2], ignore_index=True)



 11%|█         | 3/27 [00:00<00:00, 28.49it/s]

(220, 11)
(1542, 11)
(1, 11)
(5672, 11)


 22%|██▏       | 6/27 [00:01<00:04,  4.31it/s]

(1870, 11)
(673, 11)
(9595, 11)


 30%|██▉       | 8/27 [00:04<00:14,  1.32it/s]

...

(587, 11)
(315, 11)


 93%|█████████▎| 25/27 [01:02<00:02,  1.18s/it]

(13400, 11)


100%|██████████| 27/27 [01:08<00:00,  2.53s/it]

(345, 11)


In [108]:
data = imp_df
data.head()

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPresentWeatherType,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindSpeed,HourlyPresentWeatherType
0,-36.579568,-25.0,8.0,NaN,29.80,0.0,0.0,NaN,NaN,NaN,NaN,52.092749
1,-17.000000,-14.0,8.0,NaN,29.80,0.0,0.0,NaN,NaN,NaN,NaN,86.000000
2,1.342155,6.0,8.0,NaN,29.76,0.0,0.0,NaN,NaN,NaN,NaN,81.230624
3,14.000000,15.0,8.0,NaN,29.73,0.0,0.0,NaN,NaN,NaN,NaN,95.000000
4,-22.000000,-20.0,8.0,NaN,29.80,0.0,0.0,NaN,NaN,NaN,NaN,90.000000


In [109]:
data.iloc[:, 3] = data.iloc[:, 11]
data.head()

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPresentWeatherType,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindSpeed,HourlyPresentWeatherType
0,-36.579568,-25.0,8.0,52.092749,29.80,0.0,0.0,NaN,NaN,NaN,NaN,52.092749
1,-17.000000,-14.0,8.0,86.000000,29.80,0.0,0.0,NaN,NaN,NaN,NaN,86.000000
2,1.342155,6.0,8.0,81.230624,29.76,0.0,0.0,NaN,NaN,NaN,NaN,81.230624
3,14.000000,15.0,8.0,95.000000,29.73,0.0,0.0,NaN,NaN,NaN,NaN,95.000000
4,-22.000000,-20.0,8.0,90.000000,29.80,0.0,0.0,NaN,NaN,NaN,NaN,90.000000


In [110]:
data = data.iloc[:, :-1]

In [111]:
data.tail()

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPresentWeatherType,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindSpeed
103863,64.0,76.0,8.0,9.0,68.0,29.87,0.0,0.0,NaN,NaN,NaN
103864,69.0,76.0,8.0,9.0,81.0,29.83,0.0,0.0,NaN,NaN,NaN
103865,65.0,67.0,8.0,9.0,96.0,29.89,0.0,0.0,NaN,NaN,NaN
103866,65.0,69.0,8.0,9.0,87.0,29.92,0.0,0.0,NaN,NaN,NaN
103867,43.0,44.0,8.0,9.0,97.0,29.98,0.0,0.0,NaN,NaN,NaN


In [112]:
fin_df=df
fin_df.head()

,STATION,ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPresentWeatherType,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindSpeed,YEAR,MONTH,DAY,HOUR,MINUTES
0,42030099999,3066.0,NaN,NaN,-25.0,8,NaN,NaN,29.80,NaN,NaN,0.0,0.0,1944,1,2,8,30
1,42030099999,3066.0,NaN,-17.0,-14.0,8,NaN,86.0,29.80,NaN,NaN,0.0,0.0,1944,1,4,8,30
2,42030099999,3066.0,NaN,NaN,6.0,8,NaN,NaN,29.76,NaN,NaN,0.0,0.0,1944,1,5,8,30
3,42030099999,3066.0,NaN,14.0,15.0,8,NaN,95.0,29.73,NaN,NaN,0.0,0.0,1944,1,9,8,30
4,42030099999,3066.0,NaN,-22.0,-20.0,8,NaN,90.0,29.80,NaN,NaN,0.0,0.0,1944,1,10,8,30


In [113]:
import pandas as pd

# Assuming you have two DataFrames, fin_df and data
# Replace 'column_names' with the actual list of column names you want to update
column_names = ['HourlyAltimeterSetting', 'HourlyDewPointTemperature', 'HourlyDryBulbTemperature','HourlyPresentWeatherType ', 'HourlyPressureTendency', 
       'HourlyRelativeHumidity', 'HourlySeaLevelPressure', 'HourlyStationPressure', 'HourlyWetBulbTemperature', 
       'HourlyWindDirection']
# Create a copy of fin_df to preserve the original data
fin_df_copy = fin_df.copy()

# Replace specific columns in fin_df_copy with columns from data based on column names
for column_name in column_names:
    if column_name in data.columns:
        fin_df_copy[column_name] = data[column_name]

# Now fin_df_copy contains the updated columns from data that match the names in 'column_names'


In [114]:
fin_df_copy

,STATION,ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPresentWeatherType,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindSpeed,YEAR,MONTH,DAY,HOUR,MINUTES,HourlyPresentWeatherType
0,42030099999,3066.0,-36.579568,-25.0,8.0,8,29.80,0.00,0.0,NaN,NaN,NaN,0.0,1944,1,2,8,30,52.092749
1,42030099999,3066.0,-17.000000,-14.0,8.0,8,29.80,0.00,0.0,NaN,NaN,NaN,0.0,1944,1,4,8,30,86.000000
2,42030099999,3066.0,1.342155,6.0,8.0,8,29.76,0.00,0.0,NaN,NaN,NaN,0.0,1944,1,5,8,30,81.230624
3,42030099999,3066.0,14.000000,15.0,8.0,8,29.73,0.00,0.0,NaN,NaN,NaN,0.0,1944,1,9,8,30,95.000000
4,42030099999,3066.0,-22.000000,-20.0,8.0,8,29.80,0.00,0.0,NaN,NaN,NaN,0.0,1944,1,10,8,30,90.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103863,42672099999,440.0,64.000000,76.0,8.0,8,68.00,29.87,0.0,0.0,NaN,NaN,0.0,2021,1,5,17,30,9.000000
103864,42672099999,440.0,69.000000,76.0,8.0,8,81.00,29.83,0.0,0.0,NaN,NaN,0.0,2021,1,7,17,30,9.000000
103865,42672099999,440.0,65.000000,67.0,8.0,8,96.00,29.89,0.0,0.0,NaN,NaN,0.0,2021,1,9,17,30,9.000000
103866,42672099999,440.0,65.000000,69.0,8.0,8,87.00,29.92,0.0,0.0,NaN,NaN,0.0,2021,1,11,17,30,9.000000


In [115]:
fin_df = pd.DataFrame(fin_df_copy)


In [116]:
fin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103868 entries, 0 to 103867
Data columns (total 19 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   STATION                    103868 non-null  int64  
 1   ELEVATION                  103868 non-null  float64
 2   HourlyAltimeterSetting     103868 non-null  float64
 3   HourlyDewPointTemperature  103868 non-null  float64
 4   HourlyDryBulbTemperature   103868 non-null  float64
 5   HourlyPresentWeatherType   103868 non-null  int64  
 6   HourlyPressureTendency     103867 non-null  float64
 7   HourlyRelativeHumidity     103867 non-null  float64
 8   HourlySeaLevelPressure     103866 non-null  float64
 9   HourlyStationPressure      103298 non-null  float64
 10  HourlyWetBulbTemperature   97990 non-null   float64
 11  HourlyWindDirection        92832 non-null   float64
 12  HourlyWindSpeed            102673 non-null  float64
 13  YEAR                       10

In [119]:
fin_df=fin_df.iloc[:, :-1]
fin_df.isna().sum()

STATION                          0
ELEVATION                        0
HourlyAltimeterSetting           0
HourlyDewPointTemperature        0
HourlyDryBulbTemperature         0
HourlyPresentWeatherType         0
HourlyPressureTendency           1
HourlyRelativeHumidity           1
HourlySeaLevelPressure           2
HourlyStationPressure          570
HourlyWetBulbTemperature      5878
HourlyWindDirection          11036
HourlyWindSpeed               1195
YEAR                             0
MONTH                            0
DAY                              0
HOUR                             0
MINUTES                          0
dtype: int64

In [118]:
df.info()

# df.drop('SECONDS', axis=1,inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103868 entries, 0 to 103867
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   STATION                    103868 non-null  int64  
 1   ELEVATION                  103868 non-null  float64
 2   HourlyAltimeterSetting     24086 non-null   float64
 3   HourlyDewPointTemperature  101938 non-null  float64
 4   HourlyDryBulbTemperature   102611 non-null  float64
 5   HourlyPresentWeatherType   103868 non-null  int64  
 6   HourlyPressureTendency     65284 non-null   float64
 7   HourlyRelativeHumidity     101881 non-null  float64
 8   HourlySeaLevelPressure     74613 non-null   float64
 9   HourlyStationPressure      2732 non-null    float64
 10  HourlyWetBulbTemperature   2715 non-null    float64
 11  HourlyWindDirection        96079 non-null   float64
 12  HourlyWindSpeed            102673 non-null  float64
 13  YEAR                       10

In [120]:
import pandas as pd

# Load your CSV file
data = fin_df

# Assuming 'HourlyRelativeHumidity' and 'HourlySeaLevelPressure' have missing values
# HourlyPressureTendency           1
# HourlyRelativeHumidity           1
# HourlySeaLevelPressure           2
# HourlyStationPressure          570
# HourlyWetBulbTemperature      5878
# HourlyWindDirection          11036
# HourlyWindSpeed               1195

# Impute missing values with the median
data['HourlyPressureTendency'].fillna(data['HourlyPressureTendency'].median(), inplace=True)
data['HourlyRelativeHumidity'].fillna(data['HourlyRelativeHumidity'].median(), inplace=True)
data['HourlySeaLevelPressure'].fillna(data['HourlySeaLevelPressure'].median(), inplace=True)
data['HourlyStationPressure'].fillna(data['HourlyStationPressure'].median(), inplace=True)
data['HourlyWindSpeed'].fillna(data['HourlyWindSpeed'].median(), inplace=True)
data['HourlyWetBulbTemperature'].fillna(data['HourlyWetBulbTemperature'].median(), inplace=True)
data['HourlyWindDirection'].fillna(data['HourlyWindDirection'].median(), inplace=True)

data.head()


,STATION,ELEVATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPresentWeatherType,HourlyPressureTendency,HourlyRelativeHumidity,HourlySeaLevelPressure,HourlyStationPressure,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindSpeed,YEAR,MONTH,DAY,HOUR,MINUTES
0,42030099999,3066.0,-36.579568,-25.0,8.0,8,29.80,0.0,0.0,29.36662,65.033116,90.0,0.0,1944,1,2,8,30
1,42030099999,3066.0,-17.000000,-14.0,8.0,8,29.80,0.0,0.0,29.36662,65.033116,90.0,0.0,1944,1,4,8,30
2,42030099999,3066.0,1.342155,6.0,8.0,8,29.76,0.0,0.0,29.36662,65.033116,90.0,0.0,1944,1,5,8,30
3,42030099999,3066.0,14.000000,15.0,8.0,8,29.73,0.0,0.0,29.36662,65.033116,90.0,0.0,1944,1,9,8,30
4,42030099999,3066.0,-22.000000,-20.0,8.0,8,29.80,0.0,0.0,29.36662,65.033116,90.0,0.0,1944,1,10,8,30


In [121]:
data.isna().sum()

STATION                      0
ELEVATION                    0
HourlyAltimeterSetting       0
HourlyDewPointTemperature    0
HourlyDryBulbTemperature     0
HourlyPresentWeatherType     0
HourlyPressureTendency       0
HourlyRelativeHumidity       0
HourlySeaLevelPressure       0
HourlyStationPressure        0
HourlyWetBulbTemperature     0
HourlyWindDirection          0
HourlyWindSpeed              0
YEAR                         0
MONTH                        0
DAY                          0
HOUR                         0
MINUTES                      0
dtype: int64

In [122]:
# Save the updated data to a new CSV file
data.to_csv('imputed_test_data.csv', index=False)  # Save to 'imputed_data.csv'